In [79]:
import sys
import os
import numpy
from util.full import matrix

**One-electron**

We need to evaluate $\langle p_{i\beta}| l^z s^z | p_{j\beta}\rangle 
=  \langle a_{i\beta}^\dagger l^z_{pq} s^z_{pq} a_{j\beta}\rangle
=  -\langle  {l_z}_{ji} (-1/2) \rangle 
=  \frac 1 2 l^z_{ji}$


Since spin-coupling is now same for all components

$ V_{ij} = \frac 1 2 \begin{pmatrix}  
-l^z_{ji} & (l^x + i l^y)_{ji} \\
(l^x - i l^y)_{ji} & l^z_{ji}
\end{pmatrix}$

**Sulphur atom**

Run Dalton

In [80]:
#!cd S && ~/dev/dalton/git/build/master/dalton -get "AOONEINT AOPROPER SIRIUS.RST AO2SOINT" hf S

mostly we will use integrals from this set. Define filenames


In [81]:
wrkdir = "../tests/S"
def filepath(f):
    return os.path.join(wrkdir, f)

In [82]:
aooneint = filepath("hf_S.AOONEINT")
aoproper = filepath("hf_S.AOPROPER")
ao2soint = filepath("hf_S.AO2SOINT")
sirius_rst = filepath("hf_S.SIRIUS.RST")

The default input gives singlet ($3p_x^2 3p_y^2$ - HF occ 32201000). 
This is triplet Sulphur ($3p_x^1 3p_y^1 3p_z^2$) - HF occ 31102000/01100000

In [83]:
#!cd S && ~/dev/dalton/git/build/master/dalton -get "SIRIUS.RST" hf3 S

Another case is relaxation, consider the ionized form(3px^2 3py^2 2pz^1)-HF occ 32200000/32201000 (for x, y we need to do inner shell hole)

In [84]:
#!cd S && ~/dev/dalton/git/build/master/dalton -get "SIRIUS.RST" hfrel S

Now the integral is in S/hf_S.AOPROPER

In [85]:
from daltools import prop, sirrst

In [86]:
angmom = prop.read('X1SPNORB', 'Y1SPNORB', 'Z1SPNORB', filename=aoproper)
angmom_nosym = prop.read('X1SPNORB', 'Y1SPNORB', 'Z1SPNORB', filename="../tests/S_nosym/hf_S.AOPROPER")

Get the p-orbitals, read from SIRRST. We investigate three different cases:

* Default input. populate $1s^2 2s^2 2p^6 3s^2 3p_x^2 3p_y^2$
* Triplet input, populate $1s^2 2s^2 2p^6 3s^2  3p_z^2 {^3}(3p_x 3p_y)^2$
* Relaxed orbitals $1s^2 2s^2 2p^5 3s^2 3p_x^2 3p_y^2  (2p^5 = 2p_x^1 2p_y^2 2p_z^1)$


In [87]:
rst = sirrst.SiriusRestart(name=sirius_rst)
cmo = rst.cmo

In [88]:
rst_nosym = sirrst.SiriusRestart(name="../tests/S_nosym/hf_S.SIRIUS.RST")
cmo_nosym = rst_nosym.cmo[0]

In [89]:

rst3 = sirrst.SiriusRestart(name=filepath("hf3_S.SIRIUS.RST"))
cmo3 = rst3.cmo

In [90]:
rstrel = sirrst.SiriusRestart(name=filepath("hfrel_S.SIRIUS.RST"))
cmorel = rstrel.cmo

In [91]:
def get_ps(mo):
    """Pick 2p-orbitals from atomic mo input, the first orbital in second, third and fifth symmetry""" 
    _px, _py, _pz = (1, 2, 4)
    return mo[_px][:, 0], mo[_py][:, 0], mo[_pz][:, 0]

px, py, pz = get_ps(cmo)
px3, py3, pz3 = get_ps(cmo3)
pxrel, pyrel, pzrel = get_ps(cmorel)

print (px, py, pz)
#print px3, py3, pz3
#print pxrel, pyrel, pzrel


 (6,) 
              Column   1
       1      1.00179538
       2      0.00115419
       3     -0.00341405
       4     -0.00112856
       5      0.00032629
 
 (6,) 
              Column   1
       1      1.00179538
       2      0.00115419
       3     -0.00341405
       4     -0.00112856
       6      0.00032629
 
 (6,) 
              Column   1
       1      0.99656939
       2     -0.00477253
       3      0.00649723
       4      0.00218673
       5      0.00041297



In [92]:
cmo0 = cmo.unblock()
cmo03 = cmo3.unblock()
cmo0rel = cmorel.unblock()

For unblocked mos the p-orbitals in this case have indices 9, 15, 23 (0-based index), for calculation without symmetry
 2, 3, 4

In [93]:
x, y, z = 9, 15, 23
p = cmo0[:, (x, y, z)]
p3 = cmo03[:, (x, y, z)]
prel = cmo0rel[:, (x, y, z)]


In [94]:
#print p, p3, prel
print(p)


 (34, 3) 
              Column   1    Column   2    Column   3
      10      1.00179538    0.00000000    0.00000000
      11      0.00115419    0.00000000    0.00000000
      12     -0.00341405    0.00000000    0.00000000
      13     -0.00112856    0.00000000    0.00000000
      14      0.00032629    0.00000000    0.00000000
      16      0.00000000    1.00179538    0.00000000
      17      0.00000000    0.00115419    0.00000000
      18      0.00000000   -0.00341405    0.00000000
      19      0.00000000   -0.00112856    0.00000000
      21      0.00000000    0.00032629    0.00000000
      24      0.00000000    0.00000000    0.99656939
      25      0.00000000    0.00000000   -0.00477253
      26      0.00000000    0.00000000    0.00649723
      27      0.00000000    0.00000000    0.00218673
      28      0.00000000    0.00000000    0.00041297



For calculation without symmetry 

In [95]:
p_nosym = cmo_nosym[:, (2, 3, 4)]

In [96]:
print (p_nosym)


 (34, 3) 
              Column   1    Column   2    Column   3
       6      0.99656944    0.00001489   -0.00000000
       7     -0.00001481    1.00179538   -0.00000029
       8      0.00000002    0.00000029    1.00179538
       9     -0.00477242    0.00000003    0.00000000
      10      0.00000006    0.00115419   -0.00000003
      11      0.00000003    0.00000003    0.00115419
      12      0.00649715   -0.00000007   -0.00000000
      13     -0.00000007   -0.00341406    0.00000003
      14     -0.00000003   -0.00000004   -0.00341406
      15      0.00218667   -0.00000000    0.00000000
      16     -0.00000003   -0.00112856    0.00000004
      17     -0.00000002   -0.00000004   -0.00112856
      28     -0.00000001    0.00031593   -0.00000001
      30      0.00000000   -0.00008157    0.00000000
      31     -0.00000001   -0.00000001   -0.00019981
      32     -0.00025289   -0.00000000   -0.00000000
      33      0.00000001   -0.00000000    0.00025796
      34      0.00032648    0.00000

In [97]:
from scipy.constants import alpha
prefactor = alpha**2/2
ls = [prefactor*p.T*M*p for M in angmom]
ls3 = [prefactor*p3.T*M*p3 for M in angmom]
lsrel = [prefactor*prel.T*M*prel for M in angmom]
lsnosym = [prefactor*p_nosym.T*M*p_nosym for M in angmom_nosym]


In [98]:
#print ls[0], lsnosym[0]

In [99]:
#print ls[1], lsnosym[1]

In [100]:
#print ls[2], lsnosym[2]

Form complex matrices

In [101]:
V = numpy.zeros((6, 6), dtype='complex', order='F')
V3 = numpy.zeros((6, 6), dtype='complex', order='F')
Vrel = numpy.zeros((6, 6), dtype='complex', order='F')


Here we set up the spin-orbit matrix over the p-hole states in $\alpha-\beta subblocks$

$ V_{ij} = \begin{pmatrix} 
\langle a^\dagger_{i\alpha} V a_{j\alpha}\rangle & \langle a^\dagger_{i\alpha} V a_{j\beta}\rangle \\
\langle a^\dagger_{i\beta} V a_{j\alpha}\rangle & \langle a^\dagger_{i\beta} V a_{j\beta}\rangle \\
\end{pmatrix}
$

 

In [102]:
def makeV(ls):
    V = numpy.zeros((6, 6), dtype='complex', order='F')
    V[:3, :3] = -ls[2]
    V[3:, 3:] = ls[2]
    V[:3, 3:] = ls[0] + 1j*ls[1]
    V[3:, :3] = ls[0] - 1j*ls[1]
    V *= 1j/2
    return V


V = makeV(ls)
V3 = makeV(ls3)
Vrel = makeV(lsrel)
Vnosym = makeV(lsnosym)

In [103]:
print(lsnosym[0], lsnosym[1], lsnosym[2])


 (3, 3) 
              Column   1    Column   2    Column   3
       1      0.00000000   -0.00000000   -0.00000050
       2      0.00000000   -0.00000000    0.03396038
       3      0.00000050   -0.03396038   -0.00000000
 
 (3, 3) 
              Column   1    Column   2    Column   3
       1      0.00000000   -0.00000001   -0.03385742
       2      0.00000001    0.00000000   -0.00000050
       3      0.03385742    0.00000050   -0.00000000
 
 (3, 3) 
              Column   1    Column   2    Column   3
       1     -0.00000000    0.03385742   -0.00000001
       2     -0.03385742    0.00000000    0.00000000



In [104]:
print(lsnosym[2][0, 1])
print(lsnosym[2][1, 0])

0.0338574173686
-0.0338574173686


In [108]:
print(Vnosym[:3, :3])

[[ 0. +5.72293577e-23j -0. -1.69287087e-02j  0. +4.68265781e-09j]
 [ 0. +1.69287087e-02j -0. -1.62820591e-23j -0. -2.49224593e-11j]
 [-0. -4.68265781e-09j  0. +2.49224593e-11j  0. +1.66035155e-33j]]


In [31]:
eigvals =  numpy.linalg.eigvals(V)
eigvals3 =  numpy.linalg.eigvals(V3)
eigvalsrel =  numpy.linalg.eigvals(Vrel)
eigvalsnosym = numpy.linalg.eigvals(Vnosym)

In [32]:
print (eigvals.real)
print (eigvals3.real)
print (eigvalsrel.real)
print (eigvalsnosym.real)

[ 0.03389175  0.03389175 -0.01698019 -0.01691156 -0.01691156 -0.01698019]
[ 0.03388538 -0.01692596 -0.01695942  0.03388538 -0.01692596 -0.01695942]
[ 0.03527365  0.03527365 -0.01762048 -0.01765318 -0.01765318 -0.01762048]
[ 0.03389175 -0.01691156 -0.01698019  0.03389175 -0.01691156 -0.01698019]


In [33]:
eV = 27.211396132

In [35]:
split = abs(eigvals[0] - .5*(eigvals[-1]+eigvals[-2]))
print (split*eV, 'eV')

1.38336267885 eV


In [36]:
split = abs(eigvals3[0] - .5*(eigvals3[-1]+eigvals3[-2]))
print (split*eV, 'eV')

1.38310259719 eV


In [37]:
split = abs(eigvalsrel[0] - .5*(eigvalsrel[-1]+eigvalsrel[2]))
print (split*eV, 'eV')

1.43932311842 eV


**Two-electron**

In [38]:
from two.twoso import fockab

The fock routine takes alpha- and beta densities

In [39]:
#print cmo

In [40]:
nocc_alpha = [3, 2, 2, 0, 1, 0, 0, 0]
nocc_beta  = [3, 2, 2, 0, 1, 0, 0, 0]
cmoa = [ c[:, :na] for c, na in zip(cmo, nocc_alpha) ]

In [41]:
#for ca in cmoa:
#    print ca

In [42]:
from daltools import dens #is there a blocked form?


Is there a blocked range selector for occupied orbitals?

In [43]:
cmo_occ_alpha = cmo.get_columns(nocc_alpha)
cmo_occ_beta = cmo.get_columns(nocc_beta)

In [44]:
Da = (cmo_occ_alpha*cmo_occ_alpha.T()).unblock()
Db = (cmo_occ_alpha*cmo_occ_alpha.T()).unblock()

Verify

In [45]:
from daltools import one


In [46]:
Sbl = one.read(filename=filepath('hf_S.AOONEINT')).unpack()
S = Sbl.unblock()
#print Sbl

Should be 16

In [48]:
print ((Da&S) + (Db&S))

16.0


In [49]:
fa, fb = fockab(Da, Db, 'x', filename=ao2soint)

In [50]:
#print fa - fb

This convention illustrates that singlet combination is zero. The fa and fb matrix should be seen as single-contraction of twoso.

In [51]:
fabs =  [fockab(Da, Db, c, filename=ao2soint) for c in "xyz"]

In [52]:
ls2 = [0.5*(fa - fb) for fa, fb in fabs]
pls2p = [prefactor*p.T*M*p for M in ls2]


In principle these should be added as is to one-electron

In [53]:
oneso = ('X1SPNORB', 'Y1SPNORB', 'Z1SPNORB')
ls1 = prop.read(*oneso, filename=aoproper)
pls1p = [prefactor*p.T*M*p for M in ls1]


In [54]:
ls = [one + two for one, two in zip(ls1, ls2)]

In [56]:
print (pls1p[0])
print (pls2p[0])


 (3, 3) 
              Column   1    Column   2    Column   3
       2      0.00000000    0.00000000    0.03385742
       3      0.00000000   -0.03385742    0.00000000


 (3, 3) 
              Column   1    Column   2    Column   3
       2      0.00000000    0.00000000   -0.00619585
       3      0.00000000    0.00619585    0.00000000



In [68]:
print (pls1p[2])


 (3, 3) 
              Column   1    Column   2    Column   3
       1      0.00000000    0.03396038    0.00000000
       2     -0.03396038    0.00000000    0.00000000



In [57]:
plsp = [prefactor*p.T*M*p for M in ls]
print (ls[0].shape)


(34, 34)


In [58]:
V1 = makeV(pls1p)
V2 = makeV(pls2p)
V = makeV(plsp)

In [59]:
print (V1[0])
print (V2[0])

[-0.00000000+0.j         -0.00000000-0.01698019j -0.00000000+0.j
  0.00000000+0.j          0.00000000+0.j          0.01692871+0.j        ]
[-0.00000000+0.j          0.00000000+0.00308022j -0.00000000+0.j
  0.00000000+0.j          0.00000000+0.j         -0.00309792+0.j        ]


In [61]:
for v in V1, V2, V:
    eigvals = numpy.linalg.eigvals(v)
    er = numpy.sort(eigvals.real)
    print (er)
    split = abs(.5*er[:2].sum() - .25*er[2:].sum())
    print (split*eV)

[-0.01698019 -0.01698019 -0.01691156 -0.01691156  0.03389175  0.03389175]
0.693081935658
[-0.00618405 -0.00618405  0.00308022  0.00308022  0.00310383  0.00310383]
0.252414969296
[-0.01389997 -0.01389997 -0.01380775 -0.01380775  0.02770772  0.02770772]
0.567356346372


A function for these steps

In [62]:
def two_p_split(cmo, occa, occb):
    occupied_alpha = cmo.get_columns(occa)
    occupied_beta = cmo.get_columns(occb)
    
    Da = (occupied_alpha*occupied_alpha.T()).unblock()
    Db = (occupied_beta*occupied_alpha.T()).unblock()
    
    ls1 = prop.read(*oneso, filename=aoproper)
    ls2 = [0.5*(fa - fb) for fa, fb in [fockab(Da, Db, c, filename=ao2soint) for c in "xyz"]]
    
    ls = [one + two for one, two in zip(ls1, ls2)]
    
    plsp = [prefactor*p.T*ls_component*p for ls_component in ls]
    
    V = makeV(plsp)
    
    eigvals = numpy.linalg.eigvals(V)
    er = numpy.sort(eigvals.real)
    
    print (er)
    split = abs(.5*er[:2].sum() - .25*er[2:].sum())
    
    return split*eV

    

In [63]:
two_p_split(cmo, nocc_alpha, nocc_beta)

[-0.01389997 -0.01389997 -0.01380775 -0.01380775  0.02770772  0.02770772]


0.56735634637152799

In [64]:
two_p_split(cmorel, nocc_alpha, nocc_beta)

[-0.01387572 -0.01387572 -0.01377816 -0.01377816  0.02765388  0.02765388]


0.56636667397647622

In [65]:
(1.1309485892685953-1.1287511649406907)/1.1287511649406907


0.0019467748040109952